##  Face recognition and distance of face from webcam using SIFT

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def sift_detector(new_image, image_template):
    # Function that compares input image to template
    # It then returns the number of SIFT matches between them
    
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    image2 = image_template
    cv2.imshow("template",image2)
    cv2.waitKey(0)
    
    # Create SIFT detector object
    sift = cv2.xfeatures2d.SURF_create()

    # Obtain the keypoints and descriptors using SIFT
    keypoints_1, descriptors_1 = sift.detectAndCompute(image1, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(image2, None)

    # Define parameters for our Flann Matcher
    
    #FLANN stands for fast library for approximate nearest neighbours
    
    #for FLANN,we need to pass two dictionaries which specifies the algorithm to be used
    
    FLANN_INDEX_KDTREE = 0
    # below line used for both SIFT and SURF
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)
    
    # Search params is used for how many times the trees in index need to traversed
    search_params = dict(checks = 100)  # high the value,more the precision
    

    # Create the Flann Matcher object
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
  # it finds all the matches with respect to two image descriptors

    # Obtain matches using K-Nearest Neighbor Method
    # the result 'matchs' is the number of similar matches found in both images
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

    # Store good matches using Lowe's ratio test
    # we need to create only good match in available matches
    
    good_matches = []
    for m,n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m) 
    #print(matches)
    
   # draw_params=dict(matchColor=(0,255,0),singlePointColor=(255,0,0),matchesMask=good_matches,flags=0)
    
    #img3=cv2.drawMatchesKnn(image1,keypoints_1,image2,keypoints_2,matches,None,**draw_params)
    #print(img3)
    #plt.imshow(img3)
    #plt.show()

    return len(good_matches)


cap = cv2.VideoCapture(0)

# Load our image template, this is our reference image
#image_template = cv2.imread('images/box_in_scene.png', 0) 
ret, frame = cap.read()
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_classifier.detectMultiScale(gray, 1.3, 5)
if faces==():
    print("nothing")

# Every face has starting left most co ordinate and its width and height
for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
    cv2.imshow('Face Detection',frame)
    cv2.waitKey(0)
print(x,y,x+w,y+h)
fr=frame[y:y+h,x:x+w]
cv2.imshow("template0",fr)
image_template=fr
cap.release()
cap = cv2.VideoCapture(0)

while True:

    # Get webcam images
    ret, frame = cap.read()

    # Get height and width of webcam frame
 

    face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces==():
        print("nothing")

    # Every face has starting left most co ordinate and its width and height
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
        cv2.imshow('Face Detection',frame)
        cv2.waitKey(0)

    frame = cv2.flip(frame,1)  # >0 0for flipping around y axis(i.e horizontal flipping)
                               # 0 for flipping around x axis(i.e vertical flipping)
                            # <0  for flipping around both axes
    
    cropped=frame[y:y+h,x:x+w]
    
    # Get number of SIFT matches
    matches = sift_detector(cropped, image_template)

    # Display status string showing the current no. of matches 
    cv2.putText(frame,str(matches),(450,450), cv2.FONT_HERSHEY_COMPLEX, 2,(0,255,0),1)
    
    # Our threshold to indicate object deteciton
    # We use 10 since the SIFT detector returns little false positves
    threshold = 10
    
    f1=125.29343344948508 # in cms
    def dist_to_cam(known_width,fl,perwidth):
        return (known_width*fl)/perwidth
    
    width=w
    known_width=21  # width of my face using tape in cms
    distance=dist_to_cam(known_width,f1,width)
    
    # If matches exceed our threshold then object has been detected
    if matches > threshold:
       # cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0), 3)
    
        cv2.putText(frame,'Object Found'+str(distance),(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
        
    cv2.imshow('Object Detector using SIFT', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
        

cap.release()
cv2.destroyAllWindows()   

C:\Users\Manideep\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


279 173 429 323


C:\Users\Manideep\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
